In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)


Using: cuda


In [2]:
train_tfms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.1,0.1,0.1,0.05),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

test_tfms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

train_ds = datasets.ImageFolder(r"C:\Users\keval\Datasets\final_phase4\train", transform=train_tfms)
test_ds  = datasets.ImageFolder(r"C:\Users\keval\Datasets\final_phase4\test", transform=test_tfms)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=2)

print(train_ds.class_to_idx)


{'ai': 0, 'real': 1}


In [3]:
pip install timm


Note: you may need to restart the kernel to use updated packages.


In [6]:
import timm
import torch.nn as nn

model = timm.create_model("efficientnet_b0", pretrained=False, num_classes=2)

model.load_state_dict(
    torch.load(
        r"C:\Users\keval\machine learning\final_dataset\train\ai\phase2_effnet.pth",
        map_location=device,
        weights_only=True
    )
)

model.to(device)


EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (aa): Identity()
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2

In [7]:
import timm
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [9]:
for p in model.parameters():
    p.requires_grad = False

In [11]:
for i in [5, 6]:
    for p in model.blocks[i].parameters():
        p.requires_grad = True

for p in model.conv_head.parameters():
    p.requires_grad = True

for p in model.classifier.parameters():
    p.requires_grad = True


In [13]:
optimizer = AdamW([
    {"params": model.classifier.parameters(), "lr": 1e-4},
    {"params": model.conv_head.parameters(), "lr": 3e-5},
    {"params": model.blocks[5].parameters(), "lr": 3e-5},
    {"params": model.blocks[6].parameters(), "lr": 3e-5},
], weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=6)

criterion = nn.CrossEntropyLoss()

In [15]:
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=0)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=0)


In [17]:
from torchvision.datasets import ImageFolder
from torchvision.datasets.folder import default_loader

class SafeImageFolder(ImageFolder):
    def __getitem__(self, index):
        try:
            return super().__getitem__(index)
        except Exception as e:
            print(f"⚠️ Skipping bad image at index {index}: {e}")
            return self.__getitem__((index + 1) % len(self))


In [19]:
train_ds = SafeImageFolder(r"C:\Users\keval\Datasets\final_phase4\train", transform=train_tfms)
test_ds  = SafeImageFolder(r"C:\Users\keval\Datasets\final_phase4\test", transform=test_tfms)


In [23]:
# =========================
# Training only
# =========================
EPOCHS = 6   


for epoch in range(EPOCHS):

    print(f"\nEpoch {epoch+1}/{EPOCHS} — Training")
    model.train()

    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        preds = outputs.argmax(1)
        train_correct += (preds == labels).sum().item()
        train_total += labels.size(0)

    avg_train_loss = train_loss / len(train_loader)
    train_acc = 100 * train_correct / train_total

    print(f"Train loss: {avg_train_loss:.4f} | Train acc: {train_acc:.2f}%")

# =========================
# Evaluation (only once)
# =========================

print("\nFinal evaluation on test set...")
model.eval()

test_correct = 0
test_total = 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        outputs = model(imgs)
        preds = outputs.argmax(1)

        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)

test_acc = 100 * test_correct / test_total
print(f"\nFinal Test Accuracy: {test_acc:.2f}%")



Epoch 1/6 — Training
Train loss: 0.5382 | Train acc: 86.08%

Epoch 2/6 — Training
Train loss: 0.2003 | Train acc: 93.89%

Epoch 3/6 — Training
Train loss: 0.1443 | Train acc: 95.57%

Epoch 4/6 — Training
Train loss: 0.0851 | Train acc: 97.08%

Epoch 5/6 — Training


KeyboardInterrupt: 

In [25]:
model.eval()

test_correct = 0
test_total = 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        preds = model(imgs).argmax(1)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)

print("Final Test Accuracy:", 100 * test_correct / test_total)


OSError: unrecognized data stream contents when reading image file

In [27]:
from PIL import Image
import os

def find_bad_images(root):
    bad = []
    for dirpath, _, filenames in os.walk(root):
        for f in filenames:
            if f.lower().endswith((".jpg", ".jpeg", ".png", ".webp")):
                path = os.path.join(dirpath, f)
                try:
                    with Image.open(path) as img:
                        img.load()  # force decode
                except Exception as e:
                    print("❌ Bad:", path, "->", e)
                    bad.append(path)
    print(f"\nFound {len(bad)} bad images.")
    return bad

bad_test = find_bad_images("final123/user/keval/test")



Found 0 bad images.


In [29]:
from torchvision.datasets import ImageFolder
from PIL import Image

class UltraSafeImageFolder(ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        try:
            with open(path, "rb") as f:
                img = Image.open(f)
                img = img.convert("RGB")
        except Exception as e:
            print(f"⚠️ Skipping {path}: {e}")
            return self.__getitem__((index + 1) % len(self))

        if self.transform is not None:
            img = self.transform(img)

        return img, target


In [31]:
test_ds = UltraSafeImageFolder(r"C:\Users\keval\Datasets\final_phase4\test", transform=test_tfms)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=0)


In [33]:
model.eval()
correct = total = 0

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        preds = model(imgs).argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print("Final Test Accuracy:", 100 * correct / total)


⚠️ Skipping C:\Users\keval\Datasets\final_phase4\test\ai\hard_ai_3354.png: unrecognized data stream contents when reading image file
Final Test Accuracy: 85.04572350726197


In [35]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

y_true = []
y_pred = []

model.eval()
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        preds = model(imgs).argmax(1).cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(labels.numpy())

print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=["AI", "Real"]))


⚠️ Skipping C:\Users\keval\Datasets\final_phase4\test\ai\hard_ai_3354.png: unrecognized data stream contents when reading image file
[[626 233]
 [ 45 955]]
              precision    recall  f1-score   support

          AI       0.93      0.73      0.82       859
        Real       0.80      0.95      0.87      1000

    accuracy                           0.85      1859
   macro avg       0.87      0.84      0.85      1859
weighted avg       0.86      0.85      0.85      1859



In [37]:
torch.save(model.state_dict(), "phase3_effnet_final.pth")
print("Model saved as phase3_effnet_final.pth")


Model saved as phase3_effnet_final.pth


In [1]:
import timm
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = timm.create_model("efficientnet_b0", pretrained=False, num_classes=2)
model.load_state_dict(torch.load("phase3_effnet_final.pth", map_location=device))
model.to(device)
model.eval()


C:\Users\keval\AppData\Local\Temp\ipykernel_17644\2323787924.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("phase3_effnet_final.pth", 

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (aa): Identity()
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2

In [29]:
from PIL import Image
from torchvision import transforms

img_path =r"C:\Users\keval\Downloads\Gemini_Generated_Image_igdmseigdmseigdm.png"
tfm = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

img = Image.open(img_path).convert("RGB")
x = tfm(img).unsqueeze(0).to(device)

with torch.no_grad():
    out = model(x)
    probs = torch.softmax(out, dim=1)
    pred = probs.argmax(1).item()

classes = ["AI", "Real"]

print("Prediction:", classes[pred])
print("Confidence:", probs[0][pred].item())


Prediction: AI
Confidence: 0.9999991655349731


In [49]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

TEST_ROOT = r"C:\Users\keval\Datasets\final_phase4\test"

test_ds = ImageFolder(TEST_ROOT, transform=test_tfms)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=0)

print("Class mapping:", test_ds.class_to_idx)


Class mapping: {'ai': 0, 'real': 1}


In [61]:
misclassified = []
idx = 0

model.eval()
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        preds = model(imgs).argmax(1).cpu()

        for i in range(len(labels)):
            true = labels[i].item()
            pred = preds[i].item()
            path, _ = test_ds.samples[idx]

            # AI = 0, Real = 1
            if true == 0 and pred == 1:
                misclassified.append(path)

            idx += 1

print(f"Found {len(misclassified)} AI images predicted as Real")


⚠️ Skipping C:\Users\keval\Datasets\final_phase4\test\ai\hard_ai_3354.png: unrecognized data stream contents when reading image file
Found 233 AI images predicted as Real


In [62]:
from torch.utils.data import DataLoader

TEST_ROOT = r"C:\Users\keval\Datasets\phase4_missed_ai\0018.jpg"

test_ds = UltraSafeImageFolder(TEST_ROOT, transform=test_tfms)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=0)

print("Total test images:", len(test_ds))


NotADirectoryError: [WinError 267] The directory name is invalid: 'C:\\Users\\keval\\Datasets\\phase4_missed_ai\\0018.jpg'

In [66]:
import torch
import timm
from PIL import Image
from torchvision import transforms

# ========================
# Load model
# ========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = timm.create_model("efficientnet_b0", pretrained=False, num_classes=2)
model.load_state_dict(torch.load("phase3_effnet_final.pth", map_location=device))
model.to(device)
model.eval()

# ========================
# Image transform
# ========================
tfm = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

# ========================
# Predict function
# ========================
def predict_image(img_path):
    img = Image.open(img_path).convert("RGB")
    x = tfm(img).unsqueeze(0).to(device)

    with torch.no_grad():
        out = model(x)
        probs = torch.softmax(out, dim=1)
        pred = probs.argmax(1).item()

    classes = ["AI", "Real"]

    print("Image:", img_path)
    print("Prediction:", classes[pred])
    print("Confidence:", round(probs[0][pred].item(), 4))

    return classes[pred], probs[0][pred].item()

# ========================
# Example usage
# ========================
predict_image(rimport torch
import timm
from PIL import Image
from torchvision import transforms

# ========================
# Load model
# ========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = timm.create_model("efficientnet_b0", pretrained=False, num_classes=2)
model.load_state_dict(torch.load("phase3_effnet_final.pth", map_location=device))
model.to(device)
model.eval()

# ========================
# Image transform
# ========================
tfm = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

# ========================
# Predict function
# ========================
def predict_image(img_path):
    img = Image.open(img_path).convert("RGB")
    x = tfm(img).unsqueeze(0).to(device)

    with torch.no_grad():
        out = model(x)
        probs = torch.softmax(out, dim=1)
        pred = probs.argmax(1).item()

    classes = ["AI", "Real"]

    print("Image:", img_path)
    print("Prediction:", classes[pred])
    print("Confidence:", round(probs[0][pred].item(), 4))

    return classes[pred], probs[0][pred].item()

# ========================
# Example usage
# ========================
predict_image(r"C:\path\to\your\image.jpg")



SyntaxError: '(' was never closed (1831905270.py, line 48)

In [68]:
with open("phase4_missed_ai.txt", "w") as f:
    for p in misclassified:
        f.write(p + "\n")

import os, shutil
OUT_DIR = r"C:\Users\keval\Datasets\phase4_missed_ai"
os.makedirs(OUT_DIR, exist_ok=True)

for p in misclassified:
    shutil.copy(p, OUT_DIR)

print("Saved and copied failed AI images.")


Saved and copied failed AI images.


In [35]:
import torch
import timm
from PIL import Image
from torchvision import transforms

# ========================
# Load model
# ========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = timm.create_model("efficientnet_b0", pretrained=False, num_classes=2)
model.load_state_dict(torch.load("phase3_effnet_final.pth", map_location=device))
model.to(device)
model.eval()

# ========================
# Image transform
# ========================
tfm = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

# ========================
# Predict function
# ========================
def predict_image(img_path):
    img = Image.open(img_path).convert("RGB")
    x = tfm(img).unsqueeze(0).to(device)

    with torch.no_grad():
        out = model(x)
        probs = torch.softmax(out, dim=1)
        pred = probs.argmax(1).item()

    classes = ["AI", "Real"]

    print("Image:", img_path)
    print("Prediction:", classes[pred])
    print("Confidence:", round(probs[0][pred].item(), 4))

    return classes[pred], probs[0][pred].item()

# ========================
# Example usage
# ========================
predict_image(r"C:\Users\keval\Downloads\test\real\5987.jpg")



C:\Users\keval\AppData\Local\Temp\ipykernel_17644\562475323.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("phase3_effnet_final.pth", 

Image: C:\Users\keval\Downloads\test\real\5987.jpg
Prediction: Real
Confidence: 0.9555


('Real', 0.9555150270462036)

In [76]:
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

def test_accuracy(model, dataset_path, transform, batch_size=32):
    ds = ImageFolder(dataset_path, transform=transform)
    loader = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=0)

    print("Testing on:", dataset_path)
    print("Class mapping:", ds.class_to_idx)

    y_true, y_pred = [], []

    model.eval()
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            preds = model(imgs).argmax(1).cpu()

            y_true.extend(labels.numpy())
            y_pred.extend(preds.numpy())

    acc = accuracy_score(y_true, y_pred)
    print("Accuracy:", round(acc*100, 2), "%")

    return acc


In [78]:
acc = test_accuracy(
    model,
    r"C:\Users\keval\Downloads\test",
    test_tfms
)


Testing on: C:\Users\keval\Downloads\test
Class mapping: {'fake': 0, 'real': 1}


C:\Users\keval\anaconda3\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
C:\Users\keval\anaconda3\Lib\site-packages\PIL\Image.py:3368: DecompressionBombWarning: Image size (143040000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
C:\Users\keval\anaconda3\Lib\site-packages\PIL\Image.py:3368: DecompressionBombWarning: Image size (121554000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


KeyboardInterrupt: 

In [80]:
import os

model_path = "phase3_effnet_final.pth"

if os.path.exists(model_path):
    print("✅ Model file exists:", model_path)
    print("Size:", round(os.path.getsize(model_path)/1024/1024, 2), "MB")
else:
    print("❌ Model file NOT found in current directory")


✅ Model file exists: phase3_effnet_final.pth
Size: 15.58 MB


In [82]:
import os
print(os.getcwd())


C:\Users\keval
